In [ ]:
import os
from dotenv import load_dotenv
from requests.auth import HTTPBasicAuth
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import contextily as ctx
from matplotlib import pyplot as plt
import h3
import calendar
import numpy as np
from scipy.ndimage import gaussian_filter

load_dotenv()

APIKEY = os.getenv('APIKEY')

In [ ]:
fetch_data = False
bbox = (46.4, 6.5, 46.6, 6.8) # Example bounding box for Lausanne area

In [ ]:
bc_dir = os.path.join("~", "OneDrive - epfl.ch", "Research IT", "Advanced Services", "0042 – Blue City", "BlueCityViz", "SP04_Waste")

In [ ]:
lausanne_districts = os.path.join(bc_dir, "Lausanne Districts.gpkg")
# Open the geopackage file
districts_gdf = gpd.read_file(lausanne_districts)

In [ ]:
def get_monthly_data(year, month, bbox, filter_name, api_key):
    """
    Fetches data from the Sparrow API for a specific month and filter within a bounding box.
    
    Args:
        year (int): The year (e.g., 2024).
        month (int): The month (1-12).
        bbox (tuple): A tuple containing (start_lat, start_lon, end_lat, end_lon).
        filter_name (str): The specific filter to query (e.g., 'co2').
        api_key (str): The API key.
        
    Returns:
        pd.DataFrame: A DataFrame containing the fetched data.
    """
    url = 'https://api.sparrow.city/get'
    headers = {'Accept': 'application/json'}
    
    # Calculate the last day of the specific month
    _, last_day = calendar.monthrange(year, month)
    
    # Format start and end dates based on API requirements
    # Ensure month and day are zero-padded
    start_date = f"{year}-{month:02d}-01T00:00:00"
    end_date = f"{year}-{month:02d}-{last_day:02d}T23:59:59"
    
    start_lat, start_lon, end_lat, end_lon = bbox
    
    params = {
        'filter': filter_name,
        'start_date': start_date,
        'end_date': end_date,
        'start_lat': start_lat,
        'start_lon': start_lon,
        'end_lat': end_lat,
        'end_lon': end_lon,
        'api_key': api_key
    }
    
    print(f"Fetching {filter_name} data for {start_date} to {end_date}...")
    
    try:
        r = requests.get(url, headers=headers, params=params)
        r.raise_for_status() # Raise error for bad status codes
        data = r.json()
        
        if 'body' in data and data['body']:
            df = pd.DataFrame(data['body'])
            df['filter'] = filter_name
            return df
        else:
            print("No data found or empty body.")
            return pd.DataFrame()
            
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return pd.DataFrame()

In [ ]:
# Loop over the 12 months of 2025 and combine the entire dataset into a single DataFrame

if fetch_data:
  all_data = []
  for month in range(1, 13):
      df_month = get_monthly_data(2024, month, bbox, 'co2', APIKEY)
      all_data.append(df_month)
      
  all_data = pd.concat(all_data, ignore_index=True)
  all_data.to_csv('sparrow_co2_2024.csv', index=False)

In [ ]:
if not fetch_data:
    all_data = pd.read_csv('sparrow_co2_2024.csv')

In [ ]:
# Convert to geodataframe based on the x and y columns
geometry = [Point(xy) for xy in zip(all_data['x'], all_data['y'])]
gdf = gpd.GeoDataFrame(all_data, geometry=geometry)
gdf.crs = "EPSG:4326"
# gdf.to_file('data.geojson', driver='GeoJSON')

In [ ]:
ax = gdf[gdf['filter'] == 'co2'].to_crs(epsg=3857).plot(figsize=(10, 10), color="red", alpha=0.5)
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.CH)
# routes.to_crs(epsg=3857).plot(ax=ax, legend=True, column="network")

plt.title("Transit Routes in Geneva Area")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
def create_hex_bins(gdf, resolution=10):
    """Create hexagonal bins for a GeoDataFrame."""
    hex_bins = []
    for idx, row in gdf.iterrows():
        # Convert coordinates to H3 index
        h3_index = h3.latlng_to_cell(row.geometry.x, row.geometry.y, resolution)
        hex_bins.append(h3_index)
    return hex_bins

# Create hex bins for each point in the GeoDataFrame
gdf['hex_bin'] = create_hex_bins(gdf)

In [ ]:
# Calculate the average value of each filter in each hex bin
hex_avg = gdf.groupby(['hex_bin', 'filter'])['v'].mean().reset_index()

In [ ]:
# Recreate the geometries
h3_geo=gpd.GeoDataFrame(data=hex_avg, geometry = hex_avg.apply(lambda x: Polygon(h3.cell_to_boundary(x.hex_bin)),axis=1),crs=4326)
h3_geo['log_v'] = h3_geo['v'].apply(lambda x: np.log(x+1) if x > 0 else 0)


In [ ]:
# Remove any values 3 IQR below Q1 or above Q3 to further remove outliers
Q1 = h3_geo['v'].quantile(0.25)
Q3 = h3_geo['v'].quantile(0.75)
IQR = Q3 - Q1
h3_geo = h3_geo[(h3_geo['v'] >= Q1 - 1.5 * IQR) & (h3_geo['v'] <= Q3 + 1.5 * IQR)]

In [ ]:
ax=h3_geo[h3_geo['filter'] == 'co2'].to_crs(epsg=3857).plot(column='v',figsize=(10, 10),alpha=0.6)
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.CH)

# add colorbar
cbar = plt.colorbar(ax.collections[0])
cbar.set_label('CO2 Emissions (g CO2 per km)')
plt.title("Average CO2 Emissions in Lausanne Area (Hexagonal Bins)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
h3_3857 = h3_geo[h3_geo['filter'] == 'co2'].to_crs(epsg=3857)
x = h3_3857.geometry.centroid.x.values
y = h3_3857.geometry.centroid.y.values
v = h3_3857['v'].values

heatmap, xedges, yedges = np.histogram2d(x, y, bins=300, weights=v)
counts, _, _ = np.histogram2d(x, y, bins=300)

# Avoid division by zero
heatmap = np.divide(heatmap, counts, out=np.zeros_like(heatmap), where=counts!=0)

sigma = 2
heatmap_smooth = gaussian_filter(heatmap, sigma=sigma)


In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

# Add basemap first so it is at the bottom
# We need to set the extent of the axes to our data first so contextily knows what to download,
# or we can pass the crs and let it handle the current view, but explicitly setting limits is safer.
ax.set_xlim(xedges[0], xedges[-1])
ax.set_ylim(yedges[0], yedges[-1])

ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.CH, zorder = 0)

# Add the heatmap with a higher zorder
# Transpose is necessary because histogram2d follows x-y convention, fitting image expectation
im = ax.imshow(heatmap_smooth.T, origin='lower', cmap='viridis', 
               extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], 
               alpha=0.5, zorder=1)

plt.colorbar(im, label='Smoothed CO2 Emissions (Gaussian)')
plt.title("Gaussian Smoothed Field of CO2 Emissions")
# plt.axis('off')
plt.show()

In [ ]:
# Read the Quartiers statistiques shapefile in the local data folder
# The file is downloadable from https://viageo.ch/donnee/telecharger/300517
quartiers_path = os.path.join(".", "data", "Quartiers statistiques.shp")
quartiers_gdf = gpd.read_file(quartiers_path).to_crs(epsg=3857)
quartiers_gdf = quartiers_gdf[quartiers_gdf['NOMQUARTIE'] != "Zones foraines"]
quartiers_gdf.head()

In [ ]:
hexagons = quartiers_gdf.to_crs(epsg=4326).geometry.apply(lambda x: h3.geo_to_cells(x, res=11))


In [ ]:
def cell_to_shapely(cell):
    coords = h3.cell_to_boundary(cell)
    flipped = tuple(coord[::-1] for coord in coords)
    return Polygon(flipped)

In [ ]:
# Perform a left join on the exploded hexagons, renaming the geometry column as hex_hash
exploded_hexagons = hexagons.explode().apply(lambda x: cell_to_shapely(x))

quartiers_hex = quartiers_gdf.merge(exploded_hexagons, left_index=True, right_index=True, how='left')
quartiers_hex = gpd.GeoDataFrame(
    data = quartiers_hex.drop(columns=['geometry_x', 'geometry_y']),
    geometry = quartiers_hex['geometry_y'],
    crs = 4326
)


In [ ]:
# Keep only unique hexagons, and sample the co2 values for each hexagon at its centroid
hex_gdf = quartiers_hex.drop_duplicates(subset='geometry')

In [ ]:
def sample_heatmap(geometry):
    """
    Samples the heatmap value at the geometry's coordinates.
    Assumes geometry is in the same CRS (EPSG:3857) as xedges/yedges.
    """
    # Get coordinates
    px = geometry.x
    py = geometry.y
    
    # Check if point is inside the heatmap bounds
    if px < xedges[0] or px > xedges[-1] or py < yedges[0] or py > yedges[-1]:
        return np.nan

    # Calculate bin indices
    # (val - min) / (max - min) * num_bins
    x_idx = int((px - xedges[0]) / (xedges[-1] - xedges[0]) * (len(xedges) - 1))
    y_idx = int((py - yedges[0]) / (yedges[-1] - yedges[0]) * (len(yedges) - 1))

    # Clip indices to ensure they are within bounds (handle edge cases)
    x_idx = min(max(x_idx, 0), heatmap_smooth.shape[0] - 1)
    y_idx = min(max(y_idx, 0), heatmap_smooth.shape[1] - 1)

    # Return value from the smoothed heatmap
    # Note: heatmap_smooth shape is (x_bins, y_bins) from histogram2d
    return heatmap_smooth[x_idx, y_idx]


In [ ]:
hex_gdf['co2_smooth'] = hex_gdf.to_crs(epsg=3857).geometry.centroid.apply(sample_heatmap)

In [ ]:
ax = hex_gdf.to_crs(epsg=3857).plot(column='co2_smooth', figsize=(10, 10), legend=True)
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.CH)